In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv', sep='\t', error_bad_lines=False, header=None)

b'Skipping line 2120260: expected 6 fields, saw 8\n'
b'Skipping line 2446318: expected 6 fields, saw 8\n'
b'Skipping line 11141081: expected 6 fields, saw 8\n'
b'Skipping line 11152099: expected 6 fields, saw 12\nSkipping line 11152402: expected 6 fields, saw 8\n'
b'Skipping line 11882087: expected 6 fields, saw 8\n'
b'Skipping line 12902539: expected 6 fields, saw 8\nSkipping line 12935044: expected 6 fields, saw 8\n'
b'Skipping line 17589539: expected 6 fields, saw 8\n'


In [3]:
df1 = pd.read_csv('./lastfm-dataset-1K/userid-profile.tsv', sep='\t', error_bad_lines=False, header=None)

In [4]:
len(df)

19098853

In [5]:
df.head()

,0,1,2,3,4,5
0,user_000001,2009-05-04T23:08:57Z,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Deep Dish,NaN,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
1,user_000001,2009-05-04T13:54:10Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Composition 0919 (Live_2009_4_15)
2,user_000001,2009-05-04T13:52:04Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc2 (Live_2009_4_15)
3,user_000001,2009-05-04T13:42:52Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Hibari (Live_2009_4_15)
4,user_000001,2009-05-04T13:42:11Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc1 (Live_2009_4_15)


In [6]:
df.isna().sum()

0          0
1          0
2     600848
3          0
4    2162719
5         12
dtype: int64

In [7]:
df.rename(columns={0: 'user_id', 1: 'timestamp', 2: 'abc', 3: 'artist', 4: 'album', 5: 'song'}, inplace=True)

In [8]:
df.drop(columns=['abc', 'album'], inplace=True)

In [9]:
test = df

In [10]:
test2 = pd.pivot_table(test, values='artist', index=['user_id', 'song', 'artist'], aggfunc=np.count_nonzero)

In [11]:
test3 = pd.DataFrame(test2)

In [12]:
test4 = test3.reset_index()

In [13]:
test4.set_index('user_id', inplace=True)

In [14]:
test4.rename(columns={'timestamp': 'playcount'}, inplace=True)

In [15]:
total_plays_df = test4.groupby('user_id').sum()

In [16]:
total_plays_df.rename(columns={'playcount': 'user_total_plays'}, inplace=True)

In [17]:
song_df = test4.join(total_plays_df)

In [18]:
song_df['score'] = song_df.playcount / song_df.user_total_plays

In [19]:
song_df.head()

,song,artist,playcount,user_total_plays,score
user_id,,,,,
user_000001,'84 Pontiac Dream,Boards Of Canada,14,16685,0.000839
user_000001,(All I Wanted Was To Make You) Feel Good,Bugge Wesseltoft,3,16685,0.000180
user_000001,(D)Ecaying Beauty,Richie Hawtin,1,16685,0.000060
user_000001,(Far From) Home,Tiga,1,16685,0.000060
user_000001,(Get Off Your) High Horse Lady,Oasis,1,16685,0.000060


In [20]:
percentile = pd.DataFrame(song_df.groupby('user_id').score.quantile(.95))

In [23]:
test = song_df.join(percentile, rsuffix='95percentile')


In [25]:
song_df_above_95 = test[test['score'] >= test['score95percentile']]
song_df_above_95.head()

,song,artist,playcount,user_total_plays,score,score95percentile
user_id,,,,,,
user_000001,15 Step,Radiohead,24,16685,0.001438,0.001139
user_000001,1:1,Wass,19,16685,0.001139,0.001139
user_000001,33 000 Honeybees,Minilogue,19,16685,0.001139,0.001139
user_000001,A Black Man In Space (Sax Mix),Gilles Peterson,21,16685,0.001259,0.001139
user_000001,A.O.R. (Feat. Lina Ohta),Towa Tei,27,16685,0.001618,0.001139


In [26]:
song_df_above_95.drop(columns=['score95percentile'], inplace=True)
song_df_above_95.head()

/Users/briansrebrenik/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,song,artist,playcount,user_total_plays,score
user_id,,,,,
user_000001,15 Step,Radiohead,24,16685,0.001438
user_000001,1:1,Wass,19,16685,0.001139
user_000001,33 000 Honeybees,Minilogue,19,16685,0.001139
user_000001,A Black Man In Space (Sax Mix),Gilles Peterson,21,16685,0.001259
user_000001,A.O.R. (Feat. Lina Ohta),Towa Tei,27,16685,0.001618


In [27]:
len(song_df_above_95)

298174

In [29]:
song_artist_list = list(zip(song_df_above_95.song, song_df_above_95.artist))

In [30]:
song_artist_list = list(set(song_artist_list))

In [31]:
len(song_artist_list)

176312

#### Spotify API

In [32]:
import requests
client_id = '8787e23c39884c2b98026feff5b7acde'
client_secret = '94d8c07f39f841739a58e5e12c9e5e86'

grant_type = 'client_credentials'

#Request based on Client Credentials Flow from https://developer.spotify.com/web-api/authorization-guide/

#Request body parameter: grant_type Value: Required. Set it to client_credentials
body_params = {'grant_type' : grant_type}

url='https://accounts.spotify.com/api/token'

response=requests.post(url, data=body_params, auth = (client_id, client_secret)) 
print(response.text)

{"access_token":"BQDqIsNyztPnZrHs7pOh_tbXlhaGGg0KtXxha-J5xBKwNAoBV6fzeZzSCDa5D4nptPI-LW3N3S4F44soUrs","token_type":"Bearer","expires_in":3600,"scope":""}


In [33]:
access_token = 'BQDqIsNyztPnZrHs7pOh_tbXlhaGGg0KtXxha-J5xBKwNAoBV6fzeZzSCDa5D4nptPI-LW3N3S4F44soUrs'
headers = {'Authorization': 'Bearer '+ access_token}

In [34]:
r = requests.get('https://api.spotify.com/v1/search?q=artist:Kendrick%20lamar%20track:humble&type=track', headers=headers)

In [35]:
data = r.json()


In [36]:
data['tracks']['items'][0]['id']

'7KXjTSCq5nL1LoYtL7XAwS'

In [37]:
from tqdm import tqdm
def get_track_ids(tracklist):
    id_list = {}
    for track in tqdm(tracklist):
        artist = track[1].replace(" ", "%20")
        song = track[0].replace(" ", "%20")
        try:
            r = requests.get(f'https://api.spotify.com/v1/search?q=artist:{artist}%20track:{song}&type=track', headers=headers)
            data = r.json()
            track_id = data['tracks']['items'][0]['id']
            id_list[track] = track_id
        except:
            id_list[track] = "NA"
    return id_list

In [ ]:
ids = get_track_ids(song_artist_list)

  1%|          | 1648/176312 [04:37<6:29:17,  7.48it/s] 

#### getting audio features from spotify API

In [ ]:
confirmed_ids = [(x,y) for x,y in ids.items() if y != 'NA']

In [ ]:
r = requests.get('https://api.spotify.com/v1/audio-features/3fKwZ4VianLX1c7XsaUlUw', headers=headers)

In [ ]:
data = r.json()

In [ ]:
def get_track_features(track_id):
    r = requests.get(f'https://api.spotify.com/v1/audio-features/{track_id}', headers=headers)
    data = r.json()
    return data

In [ ]:
features = {}
for x, y in tqdm(confirmed_ids):
    try:
        features[x] = get_track_features(y)
    except:
        features[x] = 'NA'

In [ ]:
features_df.reset_index(inplace=True)

In [ ]:
features_df.index = [x[0] for x in confirmed_ids]
features_df['spotify_id'] = [x[1] for x in confirmed_ids]

In [ ]:
features_df.columns

In [ ]:
features_df.drop(columns=['level_0', 'level_1', 'analysis_url', 'error', 'track_href', 'type', 'id', 'uri'], inplace=True)